# TEST MODE

In [1]:
TEST_MODE=True

# Сиды

In [2]:
import numpy as np
import random
import os
import torch

seed=109

os.environ['PYTHONHASHSEED']=str(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Импорты

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from transformers import AutoModel, AutoTokenizer, CLIPModel, CLIPProcessor, AutoModelForImageClassification
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms  as v2

from catboost import Pool, CatBoostRegressor

import PIL
from PIL import Image

import math

import timm
import tqdm 
from tqdm import tqdm

2025-10-08 16:16:21.382129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759940181.583934      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759940181.653297      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Загрузка данных

In [5]:
train=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
sample=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')

train_img_dir='/kaggle/input/petfinder-pawpularity-score/train'
test_img_dir='/kaggle/input/petfinder-pawpularity-score/test'

In [6]:
if TEST_MODE:
    train=train[:100]
else:
    train=train

In [7]:
train['Pawpularity']=train['Pawpularity']/100

In [8]:
train_data, eval_data=train_test_split(train, test_size=0.2, random_state=seed)

# Datasets for all

In [9]:
class PetFinderOnlyImg(Dataset):
    def __init__(self, df, img_dir, transforms, is_train):
        self.df=df
        self.img_dir=img_dir
        self.transforms=transforms
        self.is_train=is_train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        img_in_df=row['Id']
        img_root=os.path.join(self.img_dir, f'{img_in_df}.jpg')
        image=Image.open(img_root).convert('RGB')
        

        if self.transforms is not None:
            image=self.transforms(image)
        else:
            image=image
        label=torch.tensor(row['Pawpularity'], dtype=torch.float32)

        if self.is_train:
            return{
                'image': image,
                'target': label,
                'id': row['Id']
                
            }
        else:
            return{
                'image': image,
                'id': row['Id']
                
            }
            

In [10]:
train_tfms = v2.Compose([
    v2.Resize((224, 224)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
eval_tfms = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## Создание всех датасетов 

In [11]:
#train_clip_dataset=PetFinderCLIP()
#eval_clip_dataset=PetFinderCLIP()

train_img_dataset=PetFinderOnlyImg(train_data,train_img_dir, train_tfms, is_train=True)
eval_img_dataset=PetFinderOnlyImg(eval_data, train_img_dir,train_tfms,is_train=True)

#train_text_dataset=PetFinderOnlyText()
#eval_text_dataset=PetFinderOnlyText()

In [12]:
useless=['Id']

In [13]:
X_train=train_data.drop(columns=useless)
y_train=train_data['Pawpularity']

X_eval=eval_data.drop(columns=useless)
y_eval=eval_data['Pawpularity']

## Создание даталоадеров

In [14]:
train_only_img_dataloader=DataLoader(train_img_dataset, batch_size=32, shuffle=True)
eval_only_img_dataloader=DataLoader(eval_img_dataset, batch_size=32, shuffle=False)



# Models 

## Only Img

In [15]:
IMG_MODEL_NAME='resnet50'

In [16]:
model=timm.create_model(IMG_MODEL_NAME, pretrained=True, num_classes=1).to(device)

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

## Catboost over table feats 

In [17]:
catboost_only_table = CatBoostRegressor(
    iterations=5000,
    learning_rate=0.03,
    depth=8,
    l2_leaf_reg=10.0,
    loss_function='RMSE',
    eval_metric='RMSE',
    bootstrap_type='Bayesian',
    bagging_temperature=0.5,   # мягкий стохастический бутстрэп
    #rsm=0.8,                   # семплинг фич (12 фич — осторожно, но помогает от переобучения)
    random_strength=0.3,       # штраф на «слишком уверенные» сплиты
    leaf_estimation_iterations=5,
    #od_type='Iter', od_wait=200,
    random_seed=seed,
    #allow_writing_files=False,
    verbose=200,
     task_type='GPU', devices='0',  # если есть GPU — раскомментируй
)

# Catboost over table + img

# All types of training

## EPOCHS

In [18]:
EPOCHS=5

## Img only

In [19]:
criterion=nn.MSELoss(reduction='mean')
optimizer=torch.optim.AdamW(model.parameters(),lr=3e-4, weight_decay=1e-6)
#scheduler=torch.optim.CosinaAnnealingLR(optimizer, T_max=EPOCHS)

In [20]:
steps_per_epoch = len(train_only_img_dataloader)           
warmup_steps = int(0.1 * EPOCHS * steps_per_epoch)
total_steps  = EPOCHS * steps_per_epoch

from torch.optim.lr_scheduler import LambdaLR
def lr_lambda(step):
    if step < warmup_steps:
        return float(step + 1) / float(warmup_steps)
    progress = (step - warmup_steps) / float(max(1, total_steps - warmup_steps))
    return 0.5 * (1.0 + math.cos(math.pi * progress))

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

In [21]:
from collections import deque

smooth_k = 50                 # скользящее окно для сглаживания
rmse_window = deque(maxlen=smooth_k)

for epoch in range(1, EPOCHS + 1):
    model.train()
    pbar_train = tqdm(train_only_img_dataloader, desc=f"Train {epoch}/{EPOCHS}", leave=False)
    sse_train, n_train = 0.0, 0

    for step, batch in enumerate(pbar_train, 1):
        X = batch['image'].to(device, non_blocking=True)
        y = batch['target'].to(device, non_blocking=True).float().view(-1)

        optimizer.zero_grad(set_to_none=True)
        preds = model(X).squeeze().float()
        diff = preds - y

        loss = criterion(preds, y)               
        loss.backward()
        optimizer.step()
        scheduler.step()                         

        batch_rmse = torch.sqrt(torch.mean(diff * diff)).item()
        rmse_window.append(batch_rmse)
        sse_train += torch.sum(diff * diff).item()
        n_train   += y.numel()

        pbar_train.set_postfix({
            "loss": f"{loss.item():.4f}",
            "rmse_b": f"{batch_rmse:.4f}",
            "rmse_ma": f"{(sum(rmse_window)/len(rmse_window)):.4f}",
            "rmse_avg": f"{math.sqrt(sse_train / n_train):.4f}",
        })
    #train_loss = running_loss / n_train
    train_rmse = math.sqrt(sse_train / n_train)

    # -------- Validation --------
    model.eval()
    pbar_eval = tqdm(eval_only_img_dataloader, desc="Eval", leave=False)
    n_eval = 0
    sse_eval, sae_eval = 0.0, 0.0
    sum_y, sum_y2 = 0.0, 0.0  # для R²

    with torch.no_grad():
        for batch in pbar_eval:
            X = batch['image'].to(device, non_blocking=True)
            y = batch['target'].to(device, non_blocking=True).float().view(-1)

            preds = model(X).squeeze().float()
            diff = preds - y

            sse_eval += torch.sum(diff * diff).item()
            sae_eval += torch.sum(diff.abs()).item()
            n_eval  += y.numel()

            sum_y  += torch.sum(y).item()
            sum_y2 += torch.sum(y * y).item()

    val_rmse = math.sqrt(sse_eval / n_eval)
    #val_mae  = sae_eval / n_eval
    # R² = 1 - SSE/TSS, где TSS = sum((y - ȳ)^2) = sum_y2 - sum_y^2 / n
    #tss = max(1e-12, (sum_y2 - (sum_y * sum_y) / n_eval))
    #val_r2 = 1.0 - (sse_eval / tss)

    print(f"Epoch {epoch}: "
          f"Train RMSE {train_rmse:.5f} " '|| '
          f"Val RMSE {val_rmse:.5f} ")

        

        

        

        
        

Epoch 1: Train RMSE 0.28794|| Val RMSE 0.33866 


Epoch 2: Train RMSE 0.17551|| Val RMSE 0.30653 


Epoch 3: Train RMSE 0.15662|| Val RMSE 0.31390 


Epoch 4: Train RMSE 0.12333|| Val RMSE 0.31589 


Epoch 5: Train RMSE 0.12110|| Val RMSE 0.31558 


## Catboost only table

In [22]:
catboost_only_table.fit(X_train, y_train, eval_set=(X_eval, y_eval), verbose=100, early_stopping_rounds=500, use_best_model=True)

0:	learn: 0.1805033	test: 0.2737659	best: 0.2737659 (0)	total: 8.86s	remaining: 12h 18m 10s
100:	learn: 0.0344471	test: 0.0905586	best: 0.0905586 (100)	total: 11.3s	remaining: 9m 7s
200:	learn: 0.0232630	test: 0.0630909	best: 0.0630909 (200)	total: 13.3s	remaining: 5m 18s
300:	learn: 0.0222485	test: 0.0612769	best: 0.0612769 (300)	total: 14.5s	remaining: 3m 47s
400:	learn: 0.0214219	test: 0.0603479	best: 0.0602987 (360)	total: 15.6s	remaining: 2m 58s
500:	learn: 0.0186120	test: 0.0533263	best: 0.0533263 (500)	total: 16.9s	remaining: 2m 31s
600:	learn: 0.0171968	test: 0.0506858	best: 0.0506858 (600)	total: 17.9s	remaining: 2m 10s
700:	learn: 0.0156697	test: 0.0491257	best: 0.0490458 (687)	total: 19.1s	remaining: 1m 57s
800:	learn: 0.0145449	test: 0.0489784	best: 0.0488869 (788)	total: 20.2s	remaining: 1m 45s
900:	learn: 0.0117312	test: 0.0458542	best: 0.0458286 (899)	total: 21.9s	remaining: 1m 39s
1000:	learn: 0.0099993	test: 0.0445959	best: 0.0445825 (995)	total: 23.7s	remaining: 1m 34

## Catboost over table + img embeddings  

In [23]:
import torch, os, numpy as np, pandas as pd, tqdm
from PIL import Image

# важно: используй eval-трансформы (без аугментаций) те же, что были на валидации/тесте
# tfm_eval = ...

@torch.no_grad()
def extract_prelogits(m, x: torch.Tensor) -> torch.Tensor:
    # timm-модели: стандартный путь
    if hasattr(m, "forward_features") and hasattr(m, "forward_head"):
        feats = m.forward_features(x)
        emb = m.forward_head(feats, pre_logits=True)   # [B, D]
        return emb
    # запасной вариант: если нет forward_head(pre_logits=True)
    if hasattr(m, "forward_features"):
        feats = m.forward_features(x)
        # часто feats уже [B, D] (после global pool), иначе придётся спулить вручную
        if feats.ndim == 4:  # [B, C, H, W] -> GAP
            feats = feats.mean(dim=(2,3))
        return feats
    # крайний случай: берём выход модели (нежелательно, это уже логиты)
    return m(x)

@torch.no_grad()
def build_embeds_from_current_model(model, df, img_dir, tfm, batch_size=128, device=None):
    device = device or next(model.parameters()).device
    model.eval()

    ids = df['Id'].tolist()
    out = []
    for i in tqdm.trange(0, len(ids), batch_size):
        batch_ids = ids[i:i+batch_size]
        imgs = []
        for _id in batch_ids:
            img = Image.open(os.path.join(img_dir, f"{_id}.jpg")).convert("RGB")
            imgs.append(tfm(img))
        X = torch.stack(imgs).to(device, non_blocking=True)

        emb = extract_prelogits(model, X)              # [B, D]
        out.append(emb.float().cpu().numpy())

    embs = np.concatenate(out, axis=0)                # [N, D]
    cols = [f"emb_{i}" for i in range(embs.shape[1])]
    return pd.DataFrame(embs, columns=cols).assign(Id=ids)

# примеры вызова (используй твои переменные):
emb_train = build_embeds_from_current_model(model, train_data, train_img_dir, train_tfms, batch_size=32, device=device)
emb_eval  = build_embeds_from_current_model(model, eval_data,  train_img_dir,  train_tfms, batch_size=32, device=device)



100%|██████████| 1/1 [00:00<00:00,  4.64it/s]


In [25]:
# merge
train_merged = train_data.merge(emb_train, on='Id', how='left')

eval_merged = eval_data.merge(emb_eval, on='Id', how='left')


tab_cols  = [c for c in train.columns if c not in ['Id','Pawpularity']]
emb_cols  = [c for c in train_merged.columns if c.startswith('emb_')]

X_tr = train_merged[tab_cols + emb_cols]
y_tr = train_merged['Pawpularity']
X_eval = eval_merged[tab_cols + emb_cols]
y_eval = eval_merged['Pawpularity']


cb = CatBoostRegressor(
    iterations=8000, learning_rate=0.02, depth=8,
    loss_function='RMSE', eval_metric='RMSE',
    bootstrap_type='Bayesian', bagging_temperature=0.8,
    l2_leaf_reg=20.0, random_seed=seed, verbose=200,
    task_type='GPU'
)
cb.fit(X_tr, y_tr, eval_set=(X_eval, y_eval),verbose=200, early_stopping_rounds=800, use_best_model=True)



0:	learn: 0.1844612	test: 0.2791075	best: 0.2791075 (0)	total: 3.29s	remaining: 7h 18m 4s
200:	learn: 0.1207204	test: 0.2875130	best: 0.2789602 (1)	total: 26.5s	remaining: 17m 6s
400:	learn: 0.0806263	test: 0.2904624	best: 0.2789602 (1)	total: 50.3s	remaining: 15m 52s
600:	learn: 0.0571914	test: 0.2927341	best: 0.2789602 (1)	total: 1m 14s	remaining: 15m 17s
800:	learn: 0.0432820	test: 0.2944881	best: 0.2789602 (1)	total: 1m 38s	remaining: 14m 47s
bestTest = 0.2789601998
bestIteration = 1
Shrink model to first 2 iterations.


# All types of submissions 